<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [45]:
# import PyTorch
import torch

# standard DS stack
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
# embed static images in the ipynb
%matplotlib inline 

# neural network package
import torch.nn as nn 
import torch.nn.functional as F

# computer vision
import torchvision
from torchvision import transforms
from PIL import Image

# dataset loading
from torch.utils.data import Dataset, DataLoader

# convenient package for plotting loss functions
# from livelossplot import PlotLosses

import copy
# import tqdm

In [70]:
# Run script for simulated data
import importlib.util
def get_path(sys, local):
    system_path = sys
    local_path = local 
    path = system_path + local_path
    return path

path = get_path(sys=r"C:\Users\uniqu\Adaptation\github repos"
                    + "\Bioinformatics-Neural Networks for Genomic Risk",
                local="\simulate_twas.py")
exec(open(path).read())
(beta1, beta2, beta3) = (beta for beta in betas)

genos_only.shape: (5000, 100)
phenos_only.shape: (5000,)
genos_plus.shape: (500, 100)
exprs_plus.shape: (500, 10)
phenos_plus.shape: (500,)
beta0.shape (100, 10)
beta1.shape (10,)
beta2.shape (100,)


In [71]:
# turn 1D arrays to column vectors
variables = [genos_only, phenos_only, genos_plus, exprs_plus, phenos_plus]

for idx, var in enumerate(variables):
    if var.ndim == 1:
        variables[idx] = np.reshape(variables[idx], newshape=(-1,1))
[genos_only, phenos_only, genos_plus, exprs_plus, phenos_plus] = variables

betas = list(betas)
for idx, beta in enumerate(betas):
    if var.ndim == 1:
        betas[idx] = np.reshape(betas[idx], newshape=(-1,1))

print_shapes()

genos_only.shape: (5000, 100)
phenos_only.shape: (5000, 1)
genos_plus.shape: (500, 100)
exprs_plus.shape: (500, 10)
phenos_plus.shape: (500, 1)
beta0.shape (1000, 1)
beta1.shape (10, 1)
beta2.shape (100, 1)


In [72]:
# Perform train-test split
X, Y = genos_only, phenos_only
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3,
                                                    random_state=7)

In [77]:
class simulated_Dataset(Dataset): # inherit from torch's Dataset class.
    def __init__(self, train):
        # data loading
        if train == True:
            self.X = torch.from_numpy(X_train.astype(np.float32))
            self.Y = torch.from_numpy(Y_train.astype(np.float32))
        else:
            self.X = torch.from_numpy(X_test.astype(np.float32))
            self.Y = torch.from_numpy(Y_test.reshape(-1,1).astype(np.float32))

        if self.X.shape[0] == self.Y.shape[0]:
            self.n_samples = self.X.shape[0]
        else:
            raise ValueError("Shape mismatch")
        
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]
    
    def __len__(self):
        return self.n_samples

BATCH_SIZE = 60

train_set = simulated_Dataset(train=True)
test_set = simulated_Dataset(train=False)

train_dl = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(dataset=test_set, batch_size=BATCH_SIZE, shuffle=True)

for data in train_dl:
    features, labels = data
    print(features.shape, labels.shape)
    break
print(f"Shape of single feature: {data[0][0].shape}") # 1D tensor
print(f"Shape of single target: {data[1][0].shape}") # 0D tensor (scalar)

n_features = torch.flatten(data[0][0]).shape[0]

torch.Size([60, 100]) torch.Size([60, 1])
Shape of single feature: torch.Size([100])
Shape of single target: torch.Size([1])


In [ ]:
class Sequential_Net(nn.Module): # class inherits from nn.Module
    def __init__(self):
        super(Net, self).__init__() # initialize nn.Module
        # some fully connected layers w/ linear transformation
        """ nn.Linear(in_features, out_features, bias=True)
        Args:
            in_features: size of each input sample. For input shape (28, 28), 
                we would have in_features = 28 * 28 = 784
            out_features: size of each output sample.
        """
        self.fc1 = nn.Linear(n_features, 7)
        self.fc2 = nn.Linear(7, 3)
        self.fc3 = nn.Linear(3, 1)
    def forward(self, x): # defines the forward propagation
        x = F.relu(self.fc1(x)) # relu activation function
        x = F.relu(self.fc2(x)) 
        x = F.leaky_relu(self.fc3(x))
        return x



In [ ]:
network = Sequential_Net()
print(network)